<a href="https://colab.research.google.com/github/kayleefoor/Foor_DSPN_S24/blob/main/Exercise_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 15: Power analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

1. Simulating data 1/1
2. run_analysis() function 2/2
3. repeat_analysis() function 3/3
4. Testing different sample sizes 2/2
5. Reflection 1/2

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below.

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill.

`c` is reading comprehension, and `x` is word reading experience.

In [ ]:
sample_size = 100 # How many children in data set?
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0.
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
      age <- runif(n = sample_size, min = age_lo, max = age_hi)
      x <- beta_xa * age + beta_x0 + rnorm(n = sample_size, sd = sd_x)
      c <- beta_ca * age + beta_cx * x + beta_c0 + rnorm(n = sample_size, sd = sd_c)
      dat <- data.frame(age=age, x=x, c=c)

      return(dat) # it's actually bad form to have a variable named "c" in R, my bad...
}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,age,x,c
,<dbl>,<dbl>,<dbl>
1,189.9503,-23.80924,176.4429
2,121.7465,109.20696,360.5197
3,174.5330,47.31768,235.2349
4,153.5987,-44.99892,-132.4323
5,119.6865,18.46627,180.0946
6,174.5404,117.54197,590.6237


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data.

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously.

In [ ]:
install.packages("mediation")
library(mediation)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rbibutils’, ‘checkmate’, ‘Rdpack’, ‘zoo’, ‘gridExtra’, ‘htmlTable’, ‘viridis’, ‘Formula’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘RcppEigen’, ‘mvtnorm’, ‘sandwich’, ‘lpSolve’, ‘Hmisc’, ‘lme4’


Loading required package: MASS

Loading required package: Matrix

Loading required package: mvtnorm

Loading required package: sandwich

mediation: Causal Mediation Analysis
Version: 4.5.0




In [ ]:
run_analysis <- function(data) {
  fit_mediator <- lm(x~age, data=dat)
  fit_c <- lm(c~age+x, data=dat)
  fit_med_model <- mediate(fit_mediator, fit_c, treat="age", mediator="x")
  return(c(fit_med_model$d0,
           fit_med_model$d0.p,
           fit_med_model$z0,
           fit_med_model$z0.p))
}
run_analysis(dat)

[1] 1.9365095 0.0000000 0.7023275 0.0040000

---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [ ]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)

    # Start simulating
    for (i in 1:num_simulations) {
       data <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
      simouts[i, ] <- run_analysis(data)

    }

    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    ACME_cov = mean(simouts[,2] <= alpha)
    ADE_cov = mean(simouts[,4] <= alpha)

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov, p.values=simouts[,2]))
}

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01.

In [ ]:
results = repeat_analysis(num_simulations =10, alpha=0.01, sample_size, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

$ACME_cov
[1] 1

$ADE_cov
[1] 0.7

$p.values
 [1] 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.002

---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run.

In [ ]:
sample_sizes <- c(50, 75, 100, 125, 150)
results_list <- list()

for (sample_size in sample_sizes) {
  results <- repeat_analysis(10, 0.01, sample_size, age_lo, age_hi,
                            beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
  results_list[[as.character(sample_size)]] <- results
}

Print your results.

In [ ]:
head(results_list)

$`50`
$`50`$ACME_cov
[1] 1

$`50`$ADE_cov
[1] 1

$`50`$p.values
 [1] 0.002 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000


$`75`
$`75`$ACME_cov
[1] 1

$`75`$ADE_cov
[1] 0.9

$`75`$p.values
 [1] 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.002 0.000 0.000


$`100`
$`100`$ACME_cov
[1] 1

$`100`$ADE_cov
[1] 1

$`100`$p.values
 [1] 0.002 0.000 0.000 0.000 0.002 0.000 0.000 0.000 0.000 0.000


$`125`
$`125`$ACME_cov
[1] 1

$`125`$ADE_cov
[1] 1

$`125`$p.values
 [1] 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.002 0.000 0.000


$`150`
$`150`$ACME_cov
[1] 1

$`150`$ADE_cov
[1] 0.8

$`150`$p.values
 [1] 0.000 0.002 0.002 0.000 0.000 0.000 0.000 0.000 0.000 0.000

## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above?

> *I conclude that I definitely did something wrong but I cannot figure out what it is. This output does not look correct, but I no longer have the emotional bandwidth to troubleshoot. :)*
>

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> *When the data simulated indicates a strong direct relationship between the mediator and the outcome, it is likely that you will overshadow the direct effect of the primary predictor of interest on the outcome.*

**DUE:** 5pm EST, April 3, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*